# Lab | Random Forests

- For this lab, you will be using the CSV files provided in the files_for_lab folder.

### 1. Apply the Random Forests algorithm but this time only by upscaling the data using SMOTE.

### 2. Note that since SMOTE works on numerical data only, we will first encode the categorical variables in this case.